In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageSequence
import glob
import os
from tqdm import tqdm

# Проходим по каждой строке в DataFrame с моделями
for index, row in tqdm(models_df.iterrows(), total=models_df.shape[0]):
    # Извлекаем дифференциальную форму модели
    model = row['Differential Form']

    # Определяем функцию, основанную на строке модели
    # Эта функция принимает на вход аргумент a и строку модели, возвращает результат вычисления строки модели
    def fa(a, model):
        return eval(model)

    images = []  # список для сохранения изображений графиков

    Ea_values = []  # список для хранения значений Ea
    A_values = []   # список для хранения значений A

    # Проходим по каждому значению Ea в диапазоне от 5000 до 150000 с шагом 500
    for Ea in tqdm(range(5000, 220001, 1000), desc=f"Processing model {row['Model']}"):
        try:
            # Вычисляем функцию, основанную на модели и преобразуем её с учетом Ea и температуры
            func = fa(conversion[1:-1], model) * np.exp(-Ea / (R * temperature[1:-1]))
            # Вычисляем коэффициент A как отношение последнего элемента конверсии к последнему элементу функции
            A = conversion[1:-1][-1] / func.max()
            # Получаем значение подстроенной модели
            model_fit = func*A           

            # Добавляем значения Ea и A в соответствующие списки
            Ea_values.append(Ea)
            A_values.append(A)
            
            fig, axs = plt.subplots(2)

            # Первый график: func от temperature[1:-1]
            axs[0].plot(temperature[1:-1], model_fit)            
            axs[0].set_title(row['Model'])
            axs[0].text(0.05, 0.95, f'ln(A): {np.round(np.log(A),2)}, Ea: {Ea}', ha='left', va='top', 
                        transform=axs[0].transAxes, fontsize=16)
            axs[0].tick_params(axis='both', which='major', labelsize=SMALL_SIZE)
            axs[0].set_xlabel('Температура, К', fontsize=MEDIUM_SIZE)
            axs[0].set_ylabel('Конверсия', fontsize=MEDIUM_SIZE)

            # Второй график: np.log(A) от Ea
            axs[1].plot(Ea_values, [np.log(a) for a in A_values], label='log(A) vs Ea')
            axs[1].tick_params(axis='both', which='major', labelsize=SMALL_SIZE)
            axs[1].set_xlabel('Ea', fontsize=MEDIUM_SIZE)
            axs[1].set_ylabel('log(A)', fontsize=MEDIUM_SIZE)

            # Сохраняем график как изображение в папке models_gifs
            fig_name = f"models_gifs/{row['Model']}_Ea_{Ea}.png"
            plt.savefig(fig_name)
            images.append(fig_name)

            plt.close(fig)  # закрываем фигуру, чтобы она не отображалась в notebook
            
        except Exception as e:
            # Если возникает ошибка, выводим информацию о ней и прерываем текущий цикл
            print(f"Error occurred with model: {model} and Ea: {Ea}. Error: {e}")
            break  # Если произошла ошибка, прерываем цикл для этой модели и переходим к следующей модели

    # Создаем gif анимацию
    gif_name = f"models_gifs/{row['Model']}.gif"
    frames = []
    for image in images:
        with Image.open(image) as img:
            frames.append(img.copy())
    if frames:
        frames[0].save(fp=gif_name, format='GIF', append_images=frames[1:],
                    save_all=True, duration=60, loop=0)

    # Удаляем временные изображения
    for image in images:
        os.remove(image)